In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import json
import requests
from typing import Dict, List, Optional, Tuple
import sqlite3
import hashlib
import time
import re
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AdvancedStockNewsEventsFetcher:
    def __init__(self, db_path: str = "stock_news.db"):
        self.db_path = db_path
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.init_database()
        
        # News sources configuration
        self.news_sources = {
            'yahoo_finance': {
                'base_url': 'https://finance.yahoo.com/quote/{ticker}/news',
                'parser': self._parse_yahoo_news
            },
            'marketwatch': {
                'base_url': 'https://www.marketwatch.com/investing/stock/{ticker}',
                'parser': self._parse_marketwatch_news
            },
            'reuters': {
                'base_url': 'https://www.reuters.com/markets/companies/{ticker}',
                'parser': self._parse_reuters_news
            },
            'seeking_alpha': {
                'base_url': 'https://seekingalpha.com/symbol/{ticker}/news',
                'parser': self._parse_seeking_alpha_news
            }
        }
    
    def init_database(self):
        """Initialize SQLite database with required tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create news table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_articles (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT NOT NULL,
                title TEXT NOT NULL,
                url TEXT NOT NULL,
                content TEXT,
                summary TEXT,
                publisher TEXT,
                author TEXT,
                publish_date DATETIME,
                scraped_date DATETIME DEFAULT CURRENT_TIMESTAMP,
                source_site TEXT,
                content_hash TEXT UNIQUE,
                sentiment_score REAL,
                UNIQUE(url, ticker)
            )
        ''')
        
        # Create events table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS stock_events (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT NOT NULL,
                event_date DATE,
                event_type TEXT,
                event_description TEXT,
                event_value REAL,
                scraped_date DATETIME DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(ticker, event_date, event_type, event_description)
            )
        ''')
        
        # Create price movements table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS price_movements (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ticker TEXT NOT NULL,
                movement_date DATE,
                price_change_percent REAL,
                close_price REAL,
                volume INTEGER,
                movement_type TEXT,
                scraped_date DATETIME DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(ticker, movement_date)
            )
        ''')
        
        # Create indexes for better performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_news_ticker_date ON news_articles(ticker, publish_date)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_events_ticker_date ON stock_events(ticker, event_date)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_movements_ticker_date ON price_movements(ticker, movement_date)')
        
        conn.commit()
        conn.close()
    
    def get_content_hash(self, content: str) -> str:
        """Generate MD5 hash of content to detect duplicates"""
        return hashlib.md5(content.encode('utf-8')).hexdigest()
    
    def is_news_exists(self, url: str, ticker: str, content_hash: str) -> bool:
        """Check if news article already exists in database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT id FROM news_articles 
            WHERE (url = ? AND ticker = ?) OR content_hash = ?
        ''', (url, ticker, content_hash))
        
        exists = cursor.fetchone() is not None
        conn.close()
        return exists
    
    def save_news_to_db(self, news_data: Dict, ticker: str):
        """Save news article to database if it doesn't exist"""
        content_hash = self.get_content_hash(news_data.get('content', '') + news_data.get('title', ''))
        
        if self.is_news_exists(news_data.get('url', ''), ticker, content_hash):
            logger.info(f"News already exists: {news_data.get('title', 'Unknown')[:50]}...")
            return False
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        try:
            cursor.execute('''
                INSERT INTO news_articles 
                (ticker, title, url, content, summary, publisher, author, publish_date, source_site, content_hash)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                ticker,
                news_data.get('title', ''),
                news_data.get('url', ''),
                news_data.get('content', ''),
                news_data.get('summary', ''),
                news_data.get('publisher', ''),
                news_data.get('author', ''),
                news_data.get('publish_date'),
                news_data.get('source_site', ''),
                content_hash
            ))
            conn.commit()
            logger.info(f"Saved new news: {news_data.get('title', 'Unknown')[:50]}...")
            return True
        except sqlite3.IntegrityError:
            logger.info(f"Duplicate news detected: {news_data.get('title', 'Unknown')[:50]}...")
            return False
        finally:
            conn.close()
    
    def scrape_news_for_ticker(self, ticker: str, days_back: int = 30) -> int:
        """Scrape news from multiple sources for a given ticker"""
        total_new_articles = 0
        
        for source_name, source_config in self.news_sources.items():
            logger.info(f"Scraping {source_name} for {ticker}...")
            
            try:
                url = source_config['base_url'].format(ticker=ticker.upper())
                response = self.session.get(url, timeout=10)
                response.raise_for_status()
                
                # Parse news using source-specific parser
                articles = source_config['parser'](response.text, ticker, source_name)
                
                # Save new articles to database
                new_count = 0
                for article in articles:
                    if self.save_news_to_db(article, ticker):
                        new_count += 1
                
                total_new_articles += new_count
                logger.info(f"Found {len(articles)} articles from {source_name}, {new_count} were new")
                
                # Be respectful to servers
                time.sleep(1)
                
            except Exception as e:
                logger.error(f"Error scraping {source_name} for {ticker}: {str(e)}")
                continue
        
        return total_new_articles
    
    def _parse_yahoo_news(self, html: str, ticker: str, source: str) -> List[Dict]:
        """Parse Yahoo Finance news"""
        articles = []
        soup = BeautifulSoup(html, 'html.parser')
        
        # Look for news items in Yahoo Finance structure
        news_items = soup.find_all(['li', 'div'], class_=re.compile(r'js-stream-content|story|article'))
        
        for item in news_items[:10]:  # Limit to first 10 articles
            try:
                # Extract title
                title_elem = item.find(['h3', 'h4', 'a'], class_=re.compile(r'title|headline'))
                if not title_elem:
                    title_elem = item.find('a')
                
                title = title_elem.get_text(strip=True) if title_elem else ''
                
                # Extract URL
                link_elem = title_elem if title_elem and title_elem.name == 'a' else item.find('a')
                url = link_elem.get('href', '') if link_elem else ''
                if url and not url.startswith('http'):
                    url = urljoin('https://finance.yahoo.com', url)
                
                # Extract summary/content
                summary_elem = item.find(['p', 'div'], class_=re.compile(r'summary|content|description'))
                summary = summary_elem.get_text(strip=True) if summary_elem else ''
                
                # Extract date
                date_elem = item.find(['time', 'span'], class_=re.compile(r'date|time'))
                publish_date = None
                if date_elem:
                    date_text = date_elem.get_text(strip=True)
                    publish_date = self._parse_date(date_text)
                
                if title and url:
                    articles.append({
                        'title': title,
                        'url': url,
                        'content': summary,
                        'summary': summary[:200] + '...' if len(summary) > 200 else summary,
                        'publisher': 'Yahoo Finance',
                        'author': '',
                        'publish_date': publish_date,
                        'source_site': source
                    })
                    
            except Exception as e:
                logger.debug(f"Error parsing Yahoo news item: {e}")
                continue
        
        return articles
    
    def _parse_marketwatch_news(self, html: str, ticker: str, source: str) -> List[Dict]:
        """Parse MarketWatch news"""
        articles = []
        soup = BeautifulSoup(html, 'html.parser')
        
        # Look for news items in MarketWatch structure
        news_items = soup.find_all(['div', 'article'], class_=re.compile(r'article|story|news'))
        
        for item in news_items[:10]:
            try:
                # Extract title
                title_elem = item.find(['h2', 'h3', 'h4'], class_=re.compile(r'headline|title'))
                if not title_elem:
                    title_elem = item.find('a')
                
                title = title_elem.get_text(strip=True) if title_elem else ''
                
                # Extract URL
                link_elem = title_elem.find('a') if title_elem else item.find('a')
                url = link_elem.get('href', '') if link_elem else ''
                if url and not url.startswith('http'):
                    url = urljoin('https://www.marketwatch.com', url)
                
                # Extract summary
                summary_elem = item.find(['p', 'div'], class_=re.compile(r'summary|excerpt'))
                summary = summary_elem.get_text(strip=True) if summary_elem else ''
                
                if title and url:
                    articles.append({
                        'title': title,
                        'url': url,
                        'content': summary,
                        'summary': summary[:200] + '...' if len(summary) > 200 else summary,
                        'publisher': 'MarketWatch',
                        'author': '',
                        'publish_date': datetime.now().strftime('%Y-%m-%d'),
                        'source_site': source
                    })
                    
            except Exception as e:
                logger.debug(f"Error parsing MarketWatch news item: {e}")
                continue
        
        return articles
    
    def _parse_reuters_news(self, html: str, ticker: str, source: str) -> List[Dict]:
        """Parse Reuters news"""
        articles = []
        soup = BeautifulSoup(html, 'html.parser')
        
        # Look for news items in Reuters structure
        news_items = soup.find_all(['div', 'article'], class_=re.compile(r'story|article|news'))
        
        for item in news_items[:10]:
            try:
                title_elem = item.find(['h3', 'h4'], class_=re.compile(r'headline|title'))
                title = title_elem.get_text(strip=True) if title_elem else ''
                
                link_elem = item.find('a')
                url = link_elem.get('href', '') if link_elem else ''
                if url and not url.startswith('http'):
                    url = urljoin('https://www.reuters.com', url)
                
                if title and url:
                    articles.append({
                        'title': title,
                        'url': url,
                        'content': '',
                        'summary': title,
                        'publisher': 'Reuters',
                        'author': '',
                        'publish_date': datetime.now().strftime('%Y-%m-%d'),
                        'source_site': source
                    })
                    
            except Exception as e:
                logger.debug(f"Error parsing Reuters news item: {e}")
                continue
        
        return articles
    
    def _parse_seeking_alpha_news(self, html: str, ticker: str, source: str) -> List[Dict]:
        """Parse Seeking Alpha news"""
        articles = []
        soup = BeautifulSoup(html, 'html.parser')
        
        # Look for news items in Seeking Alpha structure
        news_items = soup.find_all(['article', 'div'], class_=re.compile(r'article|post|story'))
        
        for item in news_items[:10]:
            try:
                title_elem = item.find(['h2', 'h3'], class_=re.compile(r'title|headline'))
                title = title_elem.get_text(strip=True) if title_elem else ''
                
                link_elem = item.find('a')
                url = link_elem.get('href', '') if link_elem else ''
                if url and not url.startswith('http'):
                    url = urljoin('https://seekingalpha.com', url)
                
                if title and url:
                    articles.append({
                        'title': title,
                        'url': url,
                        'content': '',
                        'summary': title,
                        'publisher': 'Seeking Alpha',
                        'author': '',
                        'publish_date': datetime.now().strftime('%Y-%m-%d'),
                        'source_site': source
                    })
                    
            except Exception as e:
                logger.debug(f"Error parsing Seeking Alpha news item: {e}")
                continue
        
        return articles
    
    def _parse_date(self, date_text: str) -> str:
        """Parse various date formats to standard format"""
        try:
            # Handle relative dates like "2 hours ago", "1 day ago"
            if 'ago' in date_text.lower():
                if 'hour' in date_text or 'minute' in date_text:
                    return datetime.now().strftime('%Y-%m-%d')
                elif 'day' in date_text:
                    days = re.findall(r'\d+', date_text)
                    if days:
                        date = datetime.now() - timedelta(days=int(days[0]))
                        return date.strftime('%Y-%m-%d')
            
            # Try to parse standard date formats
            date = pd.to_datetime(date_text)
            return date.strftime('%Y-%m-%d')
        except:
            return datetime.now().strftime('%Y-%m-%d')
    
    def get_stock_data_and_news(self, ticker: str, years_back: int = 10) -> Dict:
        """Get comprehensive stock data including scraped news"""
        logger.info(f"Starting comprehensive data collection for {ticker}")
        
        # First, scrape fresh news
        new_articles_count = self.scrape_news_for_ticker(ticker, days_back=365)
        logger.info(f"Scraped {new_articles_count} new articles for {ticker}")
        
        # Get traditional yfinance data
        stock_data = self._get_yfinance_data(ticker, years_back)
        
        # Get news from database
        db_news = self._get_news_from_db(ticker, days_back=365)
        
        # Combine all data
        result = stock_data.copy()
        result['scraped_news'] = db_news
        result['scraped_news_count'] = len(db_news)
        result['new_articles_found'] = new_articles_count
        
        return result
    
    def _get_yfinance_data(self, ticker: str, years_back: int) -> Dict:
        """Get stock data from yfinance (existing functionality)"""
        try:
            stock = yf.Ticker(ticker)
            end_date = datetime.now()
            start_date = end_date - timedelta(days=years_back * 365)
            
            info = stock.info
            hist_data = stock.history(start=start_date, end=end_date)
            actions = stock.actions
            
            if not actions.empty:
                actions_filtered = actions[actions.index >= start_date.strftime('%Y-%m-%d')]
            else:
                actions_filtered = pd.DataFrame()
            
            significant_events = self._identify_significant_events(hist_data)
            
            return {
                'ticker': ticker,
                'company_name': info.get('longName', 'N/A'),
                'sector': info.get('sector', 'N/A'),
                'industry': info.get('industry', 'N/A'),
                'date_range': {
                    'start': start_date.strftime('%Y-%m-%d'),
                    'end': end_date.strftime('%Y-%m-%d')
                },
                'corporate_actions': self._format_actions(actions_filtered),
                'significant_price_events': significant_events,
                'summary_stats': {
                    'dividends_count': len(actions_filtered[actions_filtered['Dividends'] > 0]) if not actions_filtered.empty else 0,
                    'stock_splits_count': len(actions_filtered[actions_filtered['Stock Splits'] > 0]) if not actions_filtered.empty else 0,
                    'significant_events_count': len(significant_events)
                }
            }
        except Exception as e:
            logger.error(f"Error getting yfinance data: {e}")
            return {'error': str(e)}
    
    def _get_news_from_db(self, ticker: str, days_back: int = 365) -> List[Dict]:
        """Retrieve news articles from database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cutoff_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
        
        cursor.execute('''
            SELECT title, url, content, summary, publisher, author, publish_date, source_site, scraped_date
            FROM news_articles 
            WHERE ticker = ? AND (publish_date >= ? OR publish_date IS NULL)
            ORDER BY publish_date DESC, scraped_date DESC
        ''', (ticker, cutoff_date))
        
        rows = cursor.fetchall()
        conn.close()
        
        news_list = []
        for row in rows:
            news_list.append({
                'title': row[0],
                'url': row[1],
                'content': row[2],
                'summary': row[3],
                'publisher': row[4],
                'author': row[5],
                'publish_date': row[6],
                'source_site': row[7],
                'scraped_date': row[8]
            })
        
        return news_list
    
    def _format_actions(self, actions: pd.DataFrame) -> List[Dict]:
        """Format corporate actions data"""
        if actions.empty:
            return []
        
        formatted_actions = []
        for date, row in actions.iterrows():
            date_str = date.strftime('%Y-%m-%d') if hasattr(date, 'strftime') else str(date)
            
            if row['Dividends'] > 0:
                formatted_actions.append({
                    'date': date_str,
                    'type': 'Dividend',
                    'amount': float(row['Dividends'])
                })
            if row['Stock Splits'] > 0:
                formatted_actions.append({
                    'date': date_str,
                    'type': 'Stock Split',
                    'ratio': float(row['Stock Splits'])
                })
        
        return formatted_actions
    
    def _identify_significant_events(self, hist_data: pd.DataFrame, threshold: float = 0.05) -> List[Dict]:
        """Identify significant price movements"""
        if hist_data.empty:
            return []
        
        hist_data['Daily_Return'] = hist_data['Close'].pct_change()
        significant_days = hist_data[abs(hist_data['Daily_Return']) > threshold]
        
        events = []
        for date, row in significant_days.iterrows():
            date_str = date.strftime('%Y-%m-%d') if hasattr(date, 'strftime') else str(date)
            
            events.append({
                'date': date_str,
                'price_change_percent': round(float(row['Daily_Return']) * 100, 2),
                'close_price': round(float(row['Close']), 2),
                'volume': int(row['Volume']),
                'type': 'Significant Price Movement'
            })
        
        events.sort(key=lambda x: x['date'], reverse=True)
        return events[:50]
    
    def print_comprehensive_summary(self, data: Dict):
        """Print comprehensive summary including scraped news"""
        if 'error' in data:
            print(f"Error: {data['error']}")
            return
        
        print(f"\n=== Comprehensive Summary for {data['ticker']} ({data['company_name']}) ===")
        print(f"Sector: {data['sector']}")
        print(f"Industry: {data['industry']}")
        print(f"Date Range: {data['date_range']['start']} to {data['date_range']['end']}")
        
        print(f"\n=== News Summary ===")
        print(f"  • Total Scraped Articles: {data['scraped_news_count']}")
        print(f"  • New Articles Found: {data['new_articles_found']}")
        print(f"  • Corporate Actions: {data['summary_stats']['dividends_count']} dividends, {data['summary_stats']['stock_splits_count']} splits")
        print(f"  • Significant Price Events: {data['summary_stats']['significant_events_count']}")
        
        # Show recent scraped news
        if data['scraped_news']:
            print(f"\n=== Recent Scraped News (Top 5) ===")
            for i, news in enumerate(data['scraped_news'][:5]):
                print(f"{i+1}. {news['title']}")
                print(f"   Source: {news['publisher']} ({news['source_site']}) | Date: {news['publish_date']}")
                print(f"   URL: {news['url']}")
                if news['summary']:
                    print(f"   Summary: {news['summary'][:150]}...")
                print()
    
    def export_to_json(self, data: Dict, filename: str = None):
        """Export data to JSON file"""
        if filename is None:
            filename = f"{data.get('ticker', 'unknown')}_comprehensive_data_{datetime.now().strftime('%Y%m%d')}.json"
        
        # Make data JSON serializable
        json_data = self._make_json_serializable(data)
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)
        
        print(f"Data exported to {filename}")
    
    def _make_json_serializable(self, obj):
        """Convert objects to JSON serializable format"""
        if isinstance(obj, dict):
            return {key: self._make_json_serializable(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self._make_json_serializable(item) for item in obj]
        elif isinstance(obj, pd.Timestamp):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%d %H:%M:%S')
        elif hasattr(obj, 'item'):
            return obj.item()
        elif pd.isna(obj):
            return None
        else:
            return obj


# Example usage
def main():
    # Initialize the enhanced fetcher
    fetcher = AdvancedStockNewsEventsFetcher()
    
    # Example: Get comprehensive data for Apple
    ticker = "AAPL"
    
    print("=== Enhanced Stock News and Events Fetcher ===")
    print("Features:")
    print("  • Web scraping from multiple financial news sources")
    print("  • SQLite database storage with duplicate detection")
    print("  • Comprehensive stock event tracking")
    print("\nRequired packages: pip install yfinance pandas beautifulsoup4 requests")
    print(f"\nFetching comprehensive data for {ticker}...")
    
    # Get comprehensive data (includes web scraping and database storage)
    data = fetcher.get_stock_data_and_news(ticker, years_back=2)
    
    # Print comprehensive summary
    fetcher.print_comprehensive_summary(data)
    
    # Export to JSON
    fetcher.export_to_json(data)
    
    print(f"\n=== Database Status ===")
    print(f"Database location: {fetcher.db_path}")
    print("You can query the database directly using SQLite tools or pandas:")
    print("  conn = sqlite3.connect('stock_news.db')")
    print("  df = pd.read_sql('SELECT * FROM news_articles WHERE ticker = \"AAPL\"', conn)")



In [10]:
# Initialize fetcher
fetcher = AdvancedStockNewsEventsFetcher()

# Get comprehensive data (scrapes new news + gets historical data)
data = fetcher.get_stock_data_and_news("BA", years_back=2)

# Print summary
fetcher.print_comprehensive_summary(data)

# Export to JSON
fetcher.export_to_json(data)

# Query database directly
import sqlite3
conn = sqlite3.connect('stock_news.db')
df = pd.read_sql('SELECT * FROM news_articles WHERE ticker = "AAPL"', conn)

INFO:__main__:Starting comprehensive data collection for BA
INFO:__main__:Scraping yahoo_finance for BA...
ERROR:__main__:Error scraping yahoo_finance for BA: 404 Client Error: Not Found for url: https://finance.yahoo.com/quote/BA/news/
INFO:__main__:Scraping marketwatch for BA...
ERROR:__main__:Error scraping marketwatch for BA: 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/investing/stock/BA
INFO:__main__:Scraping reuters for BA...
ERROR:__main__:Error scraping reuters for BA: 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/markets/companies/BA
INFO:__main__:Scraping seeking_alpha for BA...
ERROR:__main__:Error scraping seeking_alpha for BA: 403 Client Error: Forbidden for url: https://seekingalpha.com/symbol/BA/news
INFO:__main__:Scraped 0 new articles for BA



=== Comprehensive Summary for BA (The Boeing Company) ===
Sector: Industrials
Industry: Aerospace & Defense
Date Range: 2023-06-29 to 2025-06-28

=== News Summary ===
  • Total Scraped Articles: 0
  • New Articles Found: 0
  • Corporate Actions: 0 dividends, 0 splits
  • Significant Price Events: 15
Data exported to BA_comprehensive_data_20250628.json
